![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

# Use pretrained `match_datetime` Pipeline

### Spark `2.4` and Spark NLP `2.0.0`

* DocumentAssembler
* SentenceDetector
* Tokenizer
* DateMatcher

In [1]:
import sys
sys.path.append('../../')

#Spark ML and SQL
from pyspark.ml import Pipeline, PipelineModel
from pyspark.sql.functions import array_contains
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
#Spark NLP
from sparknlp.annotator import *
from sparknlp.common import RegexRule
from sparknlp.base import DocumentAssembler, Finisher

### Let's create a Spark Session for our app

In [2]:
spark = SparkSession.builder \
    .appName("Training_SentimentDetector")\
    .master("local[*]")\
    .config("spark.driver.memory","8G")\
    .config("spark.driver.maxResultSize", "2G")\
    .config("spark.jars", "/tmp/sparknlp.jar")\
    .config("spark.driver.extraClassPath", "/tmp/sparknlp.jar")\
    .config("spark.executor.extraClassPath", "/tmp/sparknlp.jar")\
    .config("spark.kryoserializer.buffer.max", "500m")\
    .getOrCreate()

In [3]:
spark.version

'2.4.0'

This is our testing DataFrame where we get some sentences with Phone numbers. We'll use this `transform` to predict `date/time` and extract them.

In [17]:
dfTest = spark.createDataFrame([
    "Let's meet on 20th of February.",
    "Today is March 14th 2019.",
    "I would like to come over and see you in 01/02/2019."
], StringType()).toDF("text")

In [5]:
pipeline = PipelineModel.load("/tmp/match_datetime_en_1.8.0_2.4_1552577588991")

In [18]:
# Of course you can select multiple columns at the same time however, this way we see each annotator without truncating their results
pipeline.transform(dfTest).select("token.result").show(truncate=False)
pipeline.transform(dfTest).select("date").show(truncate=False)

# Print the schema of the Pipeline
pipeline.transform(dfTest).printSchema()

+------------------------------------------------------------------+
|result                                                            |
+------------------------------------------------------------------+
|[Let, 's, meet, on, 20th, of, February, .]                        |
|[Today, is, March, 14th, 2019, .]                                 |
|[I, would, like, to, come, over, and, see, you, in, 01/02/2019, .]|
+------------------------------------------------------------------+

+----------------------------------------+
|date                                    |
+----------------------------------------+
|[[date, 14, 29, 2019/02/20, [], [], []]]|
|[[date, 9, 23, 2019/03/14, [], [], []]] |
|[[date, 41, 50, 2019/01/02, [], [], []]]|
+----------------------------------------+

root
 |-- text: string (nullable = true)
 |-- document: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (